In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
df = pd.read_csv('/kaggle/input/heart-disease-data/heart_disease_uci.csv')
df.head(20)

In [ ]:
print(df.shape)

we have a 920 person and 16 attribute

In [ ]:
print(df.columns)

id: Unique id for each patient

age:Age of the patient in years

origin: place of study

sex: Male/Female

cp: chest pain type ([typical angina, atypical angina, non-anginal, asymptomatic])

trestbps: resting blood pressure (resting blood pressure (in mm Hg on admission to the hospital))

chol: serum cholesterol in mg/dl

fbs: if fasting blood sugar > 120 mg/dl

restecg: resting electrocardiographic results) -- Values: [normal, stt abnormality, lv hypertrophy]

thalach: maximum heart rate achieved

exang: exercise-induced angina (True/ False)

oldpeak: ST depression induced by exercise relative to rest

slope: the slope of the peak exercise ST segment

ca: number of major vessels (0-3) colored by fluoroscopy

thal: [normal; fixed defect; reversible defect]

num: the predicted attribute

In [ ]:
df.rename(columns={
    "cp": "chest_pain_type",
    "trestbps": "resting_blood_pressure",
    "chol": "serum_cholesterol",
    "fbs": "fasting_blood_sugar",
    "restecg": "resting_ecg_results",
    "thalch": "max_heart_rate",
    "exang": "exercise_induced_angina",
    "oldpeak": "depression_induced_by_exercise",
    "ca" : "number_of_major_vessels",
    "thal" : "thalassemia",
    "num": "heart_disease"}, inplace=True)
df.head(10)

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
df.describe(include='object')

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(['number_of_major_vessels'],inplace=True,axis=1)

In [ ]:
df['thalassemia']=df['thalassemia'].fillna(df['thalassemia'].mode()[0])

In [ ]:
df['slope']=df['slope'].fillna(df['slope'].mode()[0])

In [ ]:
df['depression_induced_by_exercise']=df['depression_induced_by_exercise'].fillna(df['depression_induced_by_exercise'].median())

In [ ]:
df=df.dropna(subset=['exercise_induced_angina'])

In [ ]:
df['max_heart_rate']=df['max_heart_rate'].fillna(df['max_heart_rate'].median())

In [ ]:
df['resting_ecg_results']=df['resting_ecg_results'].fillna(df['resting_ecg_results'].mode()[0])

In [ ]:
df['fasting_blood_sugar']=df['fasting_blood_sugar'].fillna(df['fasting_blood_sugar'].mode())

In [ ]:
df['exercise_induced_angina']=df['exercise_induced_angina'].fillna(df['exercise_induced_angina'].mode())

In [ ]:
df['serum_cholesterol']=df['serum_cholesterol'].fillna(df['serum_cholesterol'].median())

In [ ]:
df['resting_blood_pressure']=df['resting_blood_pressure'].fillna(df['resting_blood_pressure'].median())

In [ ]:

df['fasting_blood_sugar']=df['fasting_blood_sugar'].apply(lambda x:1 if x==1 else 0)

In [ ]:
df['sex'] = df['sex'].replace({'Male': 'M', 'Female': 'F'})
df['dataset'] = df['dataset'].replace({'Cleveland':'cle',
'Hungary':'Hun',
'VA Long Beach':'va',
'Switzerland':'suiss'})
df['chest_pain_type'] = df['chest_pain_type'].replace({'typical angina':'TA', 'atypical angina':'ATA','non-anginal':'NA', 'asymptomatic':'ASY'})
df['resting_ecg_results'] = df['resting_ecg_results'].replace({'lv hypertrophy': 'lv', 'normal': 'normal','st-t abnormality':'ST'})
df['exercise_induced_angina'] = df['exercise_induced_angina'].replace({'Y': 'Yes','N': 'No'})
df['thalassemia'] = df['thalassemia'].replace({'normal':'normal',
'reversable defect':'rev',
'fixed defect':'fix'})
df['slope'] = df['slope'].replace({'Up': 'upsloping', 'Flat': 'flat', 'Down': 'downsloping'})

In [ ]:
df.head(10)

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
print(df['chest_pain_type'].value_counts())

In [ ]:
print(df['thalassemia'].value_counts())

In [ ]:
print(df['slope'].value_counts())

In [ ]:
print(df['exercise_induced_angina'].value_counts())

In [ ]:
print(df['resting_ecg_results'].value_counts())

In [ ]:
print(df['fasting_blood_sugar'].value_counts())

In [ ]:
print(df['sex'].value_counts())

In [ ]:
print(df['dataset'].value_counts())

In [ ]:
print(df['resting_blood_pressure'].value_counts())

In [ ]:
df['max_heart_rate'].value_counts()

In [ ]:
print(df['heart_disease'].value_counts())

In [ ]:
df['heart_disease'] = df['heart_disease'].apply(lambda x: 1 if x > 0 else 0)


print(df['heart_disease'].value_counts())


In [ ]:
df['serum_cholesterol'].value_counts()

162 person with 0mg it doesn't make a sense


In [ ]:
df['serum_cholesterol']=df['serum_cholesterol'].replace(0,np.nan)
df['serum_cholesterol']=df['serum_cholesterol'].fillna(df['serum_cholesterol'].median())

the outliers have been treated now

In [ ]:
unique2=df[df['serum_cholesterol']>400]
print(unique2)

we leave the big values because they may help us

In [ ]:
sns.boxplot(y='serum_cholesterol',data=df)
plt.show()

The majority of patients have serum cholesterol levels between 200 and 300 mg/dl. However, we observe outliers with values below 150 mg/dl and above 340 mg/dl

In [ ]:
sns.boxplot(y='age',data=df)
plt.show()

In [ ]:
bins = [20, 30, 40, 50, 60, 70, 80]  
labels = ['20-29', '30-39', '40-49', '50-59', '60-69', '70-79']


df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)


plt.figure(figsize=(8,5))
sns.countplot(x='age_group', data=df, order=labels, palette='viridis')
plt.title('Age Distribution by Groups')
plt.xlabel('Age Group')
plt.ylabel('Count')
plt.show()

The majority of patients are seniors between 50 and 59 years old. The youngest group is under 30, while the oldest are above 70. The distribution does not show any significant outliers

In [ ]:
sns.boxplot(y='resting_blood_pressure',data=df)
plt.show()

In [ ]:
Q1 = df['resting_blood_pressure'].quantile(0.25)
Q3 = df['resting_blood_pressure'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['resting_blood_pressure'] < lower_bound) | (df['resting_blood_pressure'] > upper_bound)]
print(outliers)

one person has 0 blood pressure it's not possible

In [ ]:
df=df[df['resting_blood_pressure']!=0]

In [ ]:
sns.boxplot(y='resting_blood_pressure',data=df)
plt.show()

we treated the outlier here

In [ ]:
bins=[0,20,40,60,80,100,120,140,160,180,200,220]
labels=['0-19','20-39','40-59','60-79','80-99','100-119','120-139','140-159','160-179','180-199','200-219']
df['blood-pressure-group']=pd.cut(df['resting_blood_pressure'],bins=bins,labels=labels,right=False)
plt.figure(figsize=(15,6)) 
sns.countplot(x='blood-pressure-group',data=df,order=labels)
plt.show()

There are outliers in the resting blood pressure feature because a value of 0 is not physiologically possible. On the other hand, values above 180 mmHg are possible since some individuals can have such high blood pressure. The majority of observations fall between 120 and 139 mmHg, which is within the common range.

we treated the outlier here

In [ ]:
sns.boxplot(y='max_heart_rate',data=df)
plt.show()

The majority of patients have a maximum heart rate between 120 and 160 bpm. The minimum observed value is around 70 bpm, while the maximum exceeds 200 bpm. We can also identify some outliers with values below 70 bpm

In [ ]:
sns.countplot(x='dataset',data=df)
plt.show()

The majority of the data comes from Cleveland, while the least comes from Switzerland

In [ ]:
df['chest_pain_type'].value_counts().sort_index().plot(kind='bar', figsize=(8,4))
plt.title("Distribution of Chest Pain Types")
plt.xlabel("Chest Pain Type")
plt.ylabel("Count")
plt.xticks(rotation=0)
plt.show()

The most common chest pain type is asymptomatic, while the least common is typical angina

In [ ]:
data=df['sex'].value_counts()
plt.pie(data,labels=data.index,autopct='%0.f%%')
plt.show()

Males represent the majority of patients (78%), whereas females make up just 22%

In [ ]:
data=df['fasting_blood_sugar'].value_counts()
plt.pie(data,labels=data.index,autopct='%0.f%%')
plt.show()

In [ ]:
sns.countplot(x='fasting_blood_sugar',data=df)
plt.show()

About 86% of patients (over 700) have a blood sugar level below 120 mg/dl, while only 14% exceed this threshold

In [ ]:
sns.countplot(
    x="resting_ecg_results", 
    data=df, 
    order=df['resting_ecg_results'].value_counts().sort_index().index,
    palette="Set2"
)

plt.xlabel("Resting ECG Results")
plt.ylabel("Count")
plt.xticks(rotation=0)
plt.show()

Normal resting ECG results are the most prevalent among patients, whereas left ventricular hypertrophy and ST-T abnormalities appear at comparable but significantly lower rates

In [ ]:
sns.countplot(
    x="exercise_induced_angina", 
    data=df, 
    order=df['exercise_induced_angina'].value_counts().sort_index().index,
    palette="Set2"
)

plt.xlabel("having an induced angina while exercising")
plt.ylabel("Count")
plt.xticks(rotation=0)
plt.show()

About 500 patients did not have exercise-induced angina, while around 300 patients did

In [ ]:
plt.hist(df['depression_induced_by_exercise'], bins=20, color="skyblue", edgecolor="black")
plt.xlabel("values")
plt.ylabel("Frequency")
plt.title("Distribution of values")
plt.show()


In [ ]:
df['depression_induced_by_exercise'].value_counts()


The majority of patients had no ST depression during the stress test (oldpeak = 0), indicating the absence of ischemic abnormalities. Only a few patients showed high ST depression values above 3, or negative values, which are rare and less clinically common.

In [ ]:
data=df['slope'].value_counts()
plt.pie(data,labels=data.index,autopct='%0.f%%')
plt.show()

Regarding the slope of the peak exercise ST segment, 69% of patients show a flat slope, 23% an upsloping pattern, and only 7% a downsloping one

In [ ]:
sns.countplot(
    x="thalassemia", 
    data=df, 
    order=df['thalassemia'].value_counts().sort_index().index,
    palette="Set2"
)

plt.ylabel("Count")
plt.xticks(rotation=0)
plt.show()

About 600 patients present a normal thalassemia result, while reversible defects are less common, and fixed defects are the least frequent

In [ ]:
sns.countplot(x='heart_disease',data=df)
plt.show()
data3=df['heart_disease'].value_counts()
plt.pie(data3,labels=data3.index,autopct='%0.f%%')
plt.show()

In [ ]:
sns.countplot(
    x="heart_disease",
    hue='sex',
    data=df, 
    
  
)

plt.ylabel("Count")
plt.xticks(rotation=0)
plt.show()

In [ ]:
sns.boxplot(x='heart_disease',y='age',data=df)
plt.show()

In [ ]:
col = list(df.columns)
categorical_features = []
numerical_features = []
for i in col:
    if len(df[i].unique()) > 6:
        numerical_features.append(i)
    else:
        categorical_features.append(i)

print('Categorical Features :',*categorical_features)

print('Numerical Features :',*numerical_features)

In [ ]:
df=df.drop(columns=['id'])

In [ ]:
colors = ['#F93822','#FDD20E']
fig, ax = plt.subplots(nrows = 4,ncols = 2,figsize = (12,20))
for i in range(len(categorical_features) - 1):
    plt.subplot(4,2,i+1)
    ax = sns.countplot(x=categorical_features[i],data = df,hue = "heart_disease",palette = colors,edgecolor = 'black')
    for rect in ax.patches:
        ax.text(rect.get_x() + rect.get_width() / 2, rect.get_height() + 2, rect.get_height(), horizontalalignment='center', fontsize = 11)
    title = categorical_features[i] + ' vs heart_disease'
    plt.legend(['No Heart Disease','Heart disease'])
    plt.title(title);
    plt.tight_layout(pad=3.0)

In [ ]:
fig, ax = plt.subplots(nrows=5, ncols=1, figsize=(15,30))
plot_idx = 1

for col in numerical_features:
    if col == 'id':  
        continue
    plt.subplot(5,1,plot_idx)
    sns.boxplot(y=col, data=df, x="heart_disease", palette=colors)
    title = col + ' vs Heart Disease'
    plt.legend(['No Heart Disease','Heart Disease'])
    plt.title(title)
    plot_idx += 1

In [ ]:
fig,ax = plt.subplots(nrows = 1,ncols = 3,figsize = (15,5))
for i in range(3):
    plt.subplot(1,3,i+1)
    sns.stripplot(x = 'sex',y = numerical_features[i],data = df,hue = 'heart_disease',palette = colors);
    plt.legend(['No Heart Disease', 'Heart Disease'])
    title = numerical_features[i] + ' vs sex' 
    plt.title(title);

fig,ax = plt.subplots(nrows = 1,ncols = 2,figsize = (15,5))
for i in [-1,-2]:
    plt.subplot(1,2,-i)
    sns.stripplot(x = 'sex',y = numerical_features[i],data = df,hue = 'heart_disease',palette = colors);
    plt.legend(['No Heart Disease', 'Heart Disease'])
    title = numerical_features[i] + ' vs sex' 
    plt.title(title);

In [ ]:

fig,ax = plt.subplots(nrows = 1,ncols = 3,figsize = (15,5))
for i in range(3):
    plt.subplot(1,3,i+1)
    sns.stripplot(x = 'chest_pain_type',y = numerical_features[i],data = df,hue = 'heart_disease',palette = colors);
    plt.legend(['No Heart Disease', 'Heart Disease'])
    title = numerical_features[i] + ' vs chest pain type' 
    plt.title(title);

fig,ax = plt.subplots(nrows = 1,ncols = 2,figsize = (15,5))
for i in [-1,-2]:
    plt.subplot(1,2,-i)
    sns.stripplot(x = 'chest_pain_type',y = numerical_features[i],data = df,hue = 'heart_disease',palette = colors);
    plt.legend(['No Heart Disease', 'Heart Disease'])
    title = numerical_features[i] + ' vs chest pain type' 
    plt.title(title);

In [ ]:
fig,ax = plt.subplots(nrows = 1,ncols = 3,figsize = (15,5))
for i in range(3):
    plt.subplot(1,3,i+1)
    sns.stripplot(x = 'slope',y = numerical_features[i],data = df,hue = 'heart_disease',palette = colors);
    plt.legend(['No Heart Disease', 'Heart Disease'])
    title = numerical_features[i] + ' vs slope' 
    plt.title(title);

fig,ax = plt.subplots(nrows = 1,ncols = 2,figsize = (15,5))
for i in [-1,-2]:
    plt.subplot(1,2,-i)
    sns.stripplot(x = 'slope',y = numerical_features[i],data = df,hue = 'heart_disease',palette = colors);
    plt.legend(['No Heart Disease', 'Heart Disease'])
    title = numerical_features[i] + ' vs slope' 
    plt.title(title);

In [ ]:
fig,ax = plt.subplots(nrows = 1,ncols = 3,figsize = (15,5))
for i in range(3):
    plt.subplot(1,3,i+1)
    sns.stripplot(x = 'fasting_blood_sugar',y = numerical_features[i],data = df,hue = 'heart_disease',palette = colors);
    plt.legend(['No Heart Disease', 'Heart Disease'])
    title = numerical_features[i] + ' vs fasting blood sugar' 
    plt.title(title);

fig,ax = plt.subplots(nrows = 1,ncols = 2,figsize = (15,5))
for i in [-1,-2]:
    plt.subplot(1,2,-i)
    sns.stripplot(x = 'fasting_blood_sugar',y = numerical_features[i],data = df,hue = 'heart_disease',palette = colors);
    plt.legend(['No Heart Disease', 'Heart Disease'])
    title = numerical_features[i] + ' vs fasting blood sugar' 
    plt.title(title);

In [ ]:
fig,ax = plt.subplots(nrows = 1,ncols = 3,figsize = (15,5))
for i in range(3):
    plt.subplot(1,3,i+1)
    sns.stripplot(x = 'resting_ecg_results',y = numerical_features[i],data = df,hue = 'heart_disease',palette = colors);
    plt.legend(['No Heart Disease', 'Heart Disease'])
    title = numerical_features[i] + ' vs resting ecg' 
    plt.title(title);

fig,ax = plt.subplots(nrows = 1,ncols = 2,figsize = (15,5))
for i in [-1,-2]:
    plt.subplot(1,2,-i)
    sns.stripplot(x = 'resting_ecg_results',y = numerical_features[i],data = df,hue = 'heart_disease',palette = colors);
    plt.legend(['No Heart Disease', 'Heart Disease'])
    title = numerical_features[i] + ' vs resting ecg' 
    plt.title(title);

In [ ]:
fig,ax = plt.subplots(nrows = 1,ncols = 3,figsize = (15,5))
for i in range(3):
    plt.subplot(1,3,i+1)
    sns.stripplot(x = 'thalassemia',y = numerical_features[i],data = df,hue = 'heart_disease',palette = colors);
    plt.legend(['No Heart Disease', 'Heart Disease'])
    title = numerical_features[i] + ' vs thalassemia' 
    plt.title(title);

fig,ax = plt.subplots(nrows = 1,ncols = 2,figsize = (15,5))
for i in [-1,-2]:
    plt.subplot(1,2,-i)
    sns.stripplot(x = 'thalassemia',y = numerical_features[i],data = df,hue = 'heart_disease',palette = colors);
    plt.legend(['No Heart Disease', 'Heart Disease'])
    title = numerical_features[i] + ' vs thalassemia' 
    plt.title(title);

In [ ]:
fig,ax = plt.subplots(nrows = 1,ncols = 3,figsize = (15,5))
for i in range(3):
    plt.subplot(1,3,i+1)
    sns.stripplot(x = 'exercise_induced_angina',y = numerical_features[i],data = df,hue = 'heart_disease',palette = colors);
    plt.legend(['No Heart Disease', 'Heart Disease'])
    title = numerical_features[i] + ' vs exercise induced angina' 
    plt.title(title);

fig,ax = plt.subplots(nrows = 1,ncols = 2,figsize = (15,5))
for i in [-1,-2]:
    plt.subplot(1,2,-i)
    sns.stripplot(x = 'exercise_induced_angina',y = numerical_features[i],data = df,hue = 'heart_disease',palette = colors);
    plt.legend(['No Heart Disease', 'Heart Disease'])
    title = numerical_features[i] + ' vs exercise induced angina' 
    plt.title(title);

In [ ]:
fig,ax = plt.subplots(nrows = 1,ncols = 3,figsize = (15,5))
for i in range(3):
    plt.subplot(1,3,i+1)
    sns.stripplot(x = 'dataset',y = numerical_features[i],data = df,hue = 'heart_disease',palette = colors);
    plt.legend(['No Heart Disease', 'Heart Disease'])
    title = numerical_features[i] + ' vs dataset' 
    plt.title(title);

fig,ax = plt.subplots(nrows = 1,ncols = 2,figsize = (15,5))
for i in [-1,-2]:
    plt.subplot(1,2,-i)
    sns.stripplot(x = 'dataset',y = numerical_features[i],data = df,hue = 'heart_disease',palette = colors);
    plt.legend(['No Heart Disease', 'Heart Disease'])
    title = numerical_features[i] + ' vs dataset' 
    plt.title(title);

In [ ]:

a = 0
fig,ax = plt.subplots(nrows = 6,ncols = 2,figsize = (15,25))
for i in range(len(numerical_features)):
    for j in range(len(numerical_features)):
        if i != j and j > i:
            a += 1
            plt.subplot(5,2,a)
            sns.scatterplot(x = numerical_features[i],y = numerical_features[j],data = df,hue = 'heart_disease',palette = colors, edgecolor = 'black');
            plt.legend(['No Heart Disease', 'Heart Disease'])
            title = numerical_features[i] + ' vs ' + numerical_features[j]
            plt.title(title)